<a href="https://colab.research.google.com/github/ravikshaga/ERAV2/blob/main/S7/ERAV2_S7_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 24, rf = 5
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 22, rf = 7

        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 11, rf = 8
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=32, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 11, rf = 8

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 9, rf = 12
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 7, rf = 16

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 3, rf = 18
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 3, rf = 18
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            # nn.ReLU() NEVER!
        ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 26

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.pool2(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [3]:
class model_1_setup(nn.Module):
    def __init__(self):
        super(model_1_setup, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) # 28>28 | 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # 28 > 28 |  5
        self.pool1 = nn.MaxPool2d(2, 2) # 28 > 14 | 10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # 14> 14 | 12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #14 > 14 | 14
        self.pool2 = nn.MaxPool2d(2, 2) # 14 > 7 | 28
        self.conv5 = nn.Conv2d(256, 512, 3) # 7 > 5 | 30
        self.conv6 = nn.Conv2d(512, 1024, 3) # 5 > 3 | 32 | 3*3*1024 | 3x3x1024x10 |
        self.conv7 = nn.Conv2d(1024, 10, 3) # 3 > 1 | 34 | > 1x1x10

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        x = self.conv7(x)
        x = x.view(-1, 10) #1x1x10> 10
        return F.log_softmax(x, dim=-1)

In [4]:
class model_2_skeleton(nn.Module):
    def __init__(self):
        super(model_2_skeleton, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 24, rf = 5


        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12, rf = 6
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 10, rf = 10

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 8, rf = 14

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 4, rf = 16
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 4, rf = 16
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(4, 4), padding=0, bias=False),
            # nn.ReLU() NEVER!
        ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 28

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        # x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        # x = self.convblock6(x)
        x = self.pool2(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [5]:
class model_3_lighter_model(nn.Module):
    def __init__(self):
        super(model_3_lighter_model, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 24, rf = 5


        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12, rf = 6
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 10, rf = 10

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 8, rf = 14

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 4, rf = 16
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 2, rf = 24
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(2, 2), padding=0, bias=False),
            # nn.ReLU() NEVER!
        ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 28

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        # x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        # x = self.convblock6(x)
        x = self.pool2(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [6]:
class model_4_batchnorm(nn.Module):
    def __init__(self):
        super(model_4_batchnorm, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 24, rf = 5


        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12, rf = 6
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 10, rf = 10

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 8, rf = 14

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 4, rf = 16
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 2, rf = 24
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(2, 2), padding=0, bias=False),
            # nn.ReLU() NEVER!
        ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 28

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        # x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        # x = self.convblock6(x)
        x = self.pool2(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [7]:
class model_5_regularization(nn.Module):
    def __init__(self):
        super(model_5_regularization, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 24, rf = 5


        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12, rf = 6
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 10, rf = 10

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 8, rf = 14

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 4, rf = 16
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 2, rf = 24
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(2, 2), padding=0, bias=False),
            # nn.ReLU() NEVER!
        ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 28

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        # x = self.convblock3(x)
        x = self.dropout(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        # x = self.convblock6(x)
        # x = self.dropout(x)
        x = self.pool2(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [8]:
class model_6_gap(nn.Module):
    def __init__(self):
        super(model_6_gap, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 24, rf = 5


        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12, rf = 6
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 10, rf = 10

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 8, rf = 14

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 4, rf = 16
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 2, rf = 24
        # self.convblock8 = nn.Sequential(
        #     nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(2, 2), padding=0, bias=False),
        #     # nn.ReLU() NEVER!
        # ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 28
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=2) # 7>> 9... nn.AdaptiveAvgPool((1, 1))
        ) # output_size = 1        rf = 28
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        # x = self.convblock3(x)
        x = self.dropout(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        # x = self.convblock6(x)
        # x = self.dropout(x)
        x = self.pool2(x)
        x = self.convblock7(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [9]:
class model_7_max_capacity(nn.Module):
    def __init__(self):
        super(model_7_max_capacity, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 24, rf = 5


        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12, rf = 6
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # output_size = 10, rf = 10

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # output_size = 8, rf = 14

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 4, rf = 16
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(12),
            nn.ReLU()
        ) # output_size = 2, rf = 24

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=2) # 7>> 9... nn.AdaptiveAvgPool((1, 1))
        ) # output_size = 1        rf = 28
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU()
        ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 28
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        # x = self.convblock3(x)
        x = self.dropout(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        # x = self.convblock6(x)
        # x = self.dropout(x)
        x = self.pool2(x)
        x = self.convblock7(x)

        x = self.gap(x)
        x = self.convblock8(x)
        # print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [26]:
dropout_value = 0.01
class model_8_maxpool_loc(nn.Module):
    def __init__(self):
        super(model_8_maxpool_loc, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(dropout_value)
        ) # output_size = 26, rf = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout(dropout_value)
        ) # output_size = 24, rf = 5


        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12, rf = 6
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(dropout_value)
        ) # output_size = 10, rf = 10

        # CONVOLUTION BLOCK 2
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout(dropout_value)
        ) # output_size = 8, rf = 14

        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 4, rf = 16
        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 2, rf = 24

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=2) # 7>> 9... nn.AdaptiveAvgPool((1, 1))
        ) # output_size = 1        rf = 28
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU()
        ) # output_size = 1 4x4x10 | 4x4x10x10 | 1x1x10, rf = 28
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        # x = self.convblock3(x)
        # x = self.dropout(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        # x = self.convblock6(x)
        # x = self.dropout(x)
        x = self.pool2(x)
        x = self.convblock7(x)

        x = self.gap(x)
        x = self.convblock8(x)
        # print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [13]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = model_8_maxpool_loc().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
           Dropout-4           [-1, 10, 26, 26]               0
            Conv2d-5           [-1, 20, 24, 24]           1,800
              ReLU-6           [-1, 20, 24, 24]               0
       BatchNorm2d-7           [-1, 20, 24, 24]              40
           Dropout-8           [-1, 20, 24, 24]               0
         MaxPool2d-9           [-1, 20, 12, 12]               0
           Conv2d-10           [-1, 10, 10, 10]           1,800
             ReLU-11           [-1, 10, 10, 10]               0
      BatchNorm2d-12           [-1, 10, 10, 10]              20
          Dropout-13           [-1, 10, 10, 10]               0
           Conv2d-14             [-1, 2

In [14]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 79829095.20it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 16578033.92it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 27344942.32it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20774840.53it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [15]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
model =  model_1_setup().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.0539318211376667 Batch_id=234 Accuracy=78.65: 100%|██████████| 235/235 [00:19<00:00, 11.95it/s]



Test set: Average loss: 0.0902, Accuracy: 9717/10000 (97.17%)

EPOCH: 1


Loss=0.03953279182314873 Batch_id=234 Accuracy=97.60: 100%|██████████| 235/235 [00:18<00:00, 12.75it/s]



Test set: Average loss: 0.0487, Accuracy: 9834/10000 (98.34%)

EPOCH: 2


...


Test set: Average loss: 0.0316, Accuracy: 9911/10000 (99.11%)

EPOCH: 15


Loss=0.004107873886823654 Batch_id=234 Accuracy=99.83: 100%|██████████| 235/235 [00:18<00:00, 12.71it/s]



Test set: Average loss: 0.0332, Accuracy: 9914/10000 (99.14%)

EPOCH: 16


Loss=0.0012227685656398535 Batch_id=77 Accuracy=99.99:  33%|███▎      | 78/235 [00:05<00:11, 13.44it/s]


KeyboardInterrupt: 

In [ ]:
model =  model_2_skeleton().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.12417484074831009 Batch_id=234 Accuracy=74.50: 100%|██████████| 235/235 [00:15<00:00, 14.95it/s]



Test set: Average loss: 0.1402, Accuracy: 9606/10000 (96.06%)

EPOCH: 1


Loss=0.12930549681186676 Batch_id=234 Accuracy=96.38: 100%|██████████| 235/235 [00:15<00:00, 14.88it/s]



Test set: Average loss: 0.0968, Accuracy: 9713/10000 (97.13%)

EPOCH: 2


...


Test set: Average loss: 0.0361, Accuracy: 9881/10000 (98.81%)

EPOCH: 13


Loss=0.0057447925209999084 Batch_id=234 Accuracy=99.08: 100%|██████████| 235/235 [00:16<00:00, 14.33it/s]



Test set: Average loss: 0.0331, Accuracy: 9888/10000 (98.88%)

EPOCH: 14


Loss=0.04925854876637459 Batch_id=234 Accuracy=99.07: 100%|██████████| 235/235 [00:17<00:00, 13.25it/s]



Test set: Average loss: 0.0327, Accuracy: 9891/10000 (98.91%)



In [ ]:
model =  model_3_lighter_model().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.2373974323272705 Batch_id=234 Accuracy=57.50: 100%|██████████| 235/235 [00:19<00:00, 12.21it/s]



Test set: Average loss: 0.2178, Accuracy: 9351/10000 (93.51%)

EPOCH: 1


Loss=0.05447584390640259 Batch_id=234 Accuracy=94.75: 100%|██████████| 235/235 [00:15<00:00, 15.19it/s]



Test set: Average loss: 0.1167, Accuracy: 9625/10000 (96.25%)

EPOCH: 2


...


Test set: Average loss: 0.0440, Accuracy: 9857/10000 (98.57%)

EPOCH: 13


Loss=0.02479751594364643 Batch_id=234 Accuracy=98.70: 100%|██████████| 235/235 [00:17<00:00, 13.79it/s]



Test set: Average loss: 0.0454, Accuracy: 9855/10000 (98.55%)

EPOCH: 14


Loss=0.030619576573371887 Batch_id=234 Accuracy=98.73: 100%|██████████| 235/235 [00:16<00:00, 14.52it/s]



Test set: Average loss: 0.0444, Accuracy: 9858/10000 (98.58%)



In [ ]:
model =  model_4_batchnorm().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.09719540923833847 Batch_id=234 Accuracy=90.18: 100%|██████████| 235/235 [00:14<00:00, 16.16it/s]



Test set: Average loss: 0.0832, Accuracy: 9772/10000 (97.72%)

EPOCH: 1


Loss=0.05110184848308563 Batch_id=234 Accuracy=98.10: 100%|██████████| 235/235 [00:14<00:00, 16.14it/s]



Test set: Average loss: 0.0558, Accuracy: 9850/10000 (98.50%)

EPOCH: 2


...


Test set: Average loss: 0.0330, Accuracy: 9903/10000 (99.03%)

EPOCH: 13


Loss=0.005976589862257242 Batch_id=234 Accuracy=99.67: 100%|██████████| 235/235 [00:15<00:00, 14.81it/s]



Test set: Average loss: 0.0307, Accuracy: 9908/10000 (99.08%)

EPOCH: 14


Loss=0.004446459002792835 Batch_id=234 Accuracy=99.71: 100%|██████████| 235/235 [00:16<00:00, 14.68it/s]



Test set: Average loss: 0.0307, Accuracy: 9897/10000 (98.97%)



In [ ]:

model =  model_5_regularization().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.2151293009519577 Batch_id=234 Accuracy=89.45: 100%|██████████| 235/235 [00:19<00:00, 11.76it/s]



Test set: Average loss: 0.1419, Accuracy: 9667/10000 (96.67%)

EPOCH: 1


Loss=0.13075082004070282 Batch_id=234 Accuracy=97.90: 100%|██████████| 235/235 [00:14<00:00, 16.14it/s]



Test set: Average loss: 0.0890, Accuracy: 9766/10000 (97.66%)

EPOCH: 2


...


Test set: Average loss: 0.0366, Accuracy: 9880/10000 (98.80%)

EPOCH: 13


Loss=0.047114282846450806 Batch_id=234 Accuracy=99.45: 100%|██████████| 235/235 [00:14<00:00, 16.06it/s]



Test set: Average loss: 0.0327, Accuracy: 9898/10000 (98.98%)

EPOCH: 14


Loss=0.017176659777760506 Batch_id=234 Accuracy=99.48: 100%|██████████| 235/235 [00:15<00:00, 15.15it/s]



Test set: Average loss: 0.0320, Accuracy: 9892/10000 (98.92%)



In [ ]:

model =  model_6_gap().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.24089817702770233 Batch_id=234 Accuracy=92.17: 100%|██████████| 235/235 [00:14<00:00, 16.19it/s]



Test set: Average loss: 0.2749, Accuracy: 9685/10000 (96.85%)

EPOCH: 1


Loss=0.1604248732328415 Batch_id=234 Accuracy=98.00: 100%|██████████| 235/235 [00:14<00:00, 15.98it/s]



Test set: Average loss: 0.2002, Accuracy: 9708/10000 (97.08%)

EPOCH: 2


...


Test set: Average loss: 0.0613, Accuracy: 9873/10000 (98.73%)

EPOCH: 13


Loss=0.05622607469558716 Batch_id=234 Accuracy=99.38: 100%|██████████| 235/235 [00:14<00:00, 15.75it/s]



Test set: Average loss: 0.0520, Accuracy: 9892/10000 (98.92%)

EPOCH: 14


Loss=0.018960118293762207 Batch_id=234 Accuracy=99.37: 100%|██████████| 235/235 [00:15<00:00, 15.48it/s]



Test set: Average loss: 0.0480, Accuracy: 9897/10000 (98.97%)



In [ ]:

model =  model_7_max_capacity().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.22025322914123535 Batch_id=234 Accuracy=82.37: 100%|██████████| 235/235 [00:19<00:00, 12.34it/s]



Test set: Average loss: 0.2103, Accuracy: 9682/10000 (96.82%)

EPOCH: 1


Loss=0.04802678897976875 Batch_id=234 Accuracy=97.91: 100%|██████████| 235/235 [00:16<00:00, 14.48it/s]



Test set: Average loss: 0.1122, Accuracy: 9780/10000 (97.80%)

EPOCH: 2


...


Test set: Average loss: 0.0611, Accuracy: 9832/10000 (98.32%)

EPOCH: 13


Loss=0.03127061948180199 Batch_id=234 Accuracy=99.45: 100%|██████████| 235/235 [00:16<00:00, 14.19it/s]



Test set: Average loss: 0.0399, Accuracy: 9892/10000 (98.92%)

EPOCH: 14


Loss=0.018751196563243866 Batch_id=234 Accuracy=99.52: 100%|██████████| 235/235 [00:14<00:00, 15.67it/s]



Test set: Average loss: 0.0403, Accuracy: 9877/10000 (98.77%)



In [16]:
model =  model_8_maxpool_loc().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.015783540904521942 Batch_id=234 Accuracy=93.23: 100%|██████████| 235/235 [00:15<00:00, 15.27it/s]



Test set: Average loss: 0.0537, Accuracy: 9841/10000 (98.41%)

EPOCH: 1


Loss=0.07585760205984116 Batch_id=234 Accuracy=98.15: 100%|██████████| 235/235 [00:16<00:00, 14.43it/s]



Test set: Average loss: 0.0462, Accuracy: 9860/10000 (98.60%)

EPOCH: 2


...


Test set: Average loss: 0.0221, Accuracy: 9928/10000 (99.28%)

EPOCH: 13


Loss=0.007819745689630508 Batch_id=234 Accuracy=99.32: 100%|██████████| 235/235 [00:15<00:00, 15.06it/s]



Test set: Average loss: 0.0240, Accuracy: 9919/10000 (99.19%)

EPOCH: 14


Loss=0.004103632178157568 Batch_id=234 Accuracy=99.41: 100%|██████████| 235/235 [00:16<00:00, 14.02it/s]



Test set: Average loss: 0.0258, Accuracy: 9917/10000 (99.17%)



In [17]:
model =  model_8_maxpool_loc().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.060612231492996216 Batch_id=234 Accuracy=94.10: 100%|██████████| 235/235 [00:15<00:00, 15.38it/s]



Test set: Average loss: 0.0521, Accuracy: 9839/10000 (98.39%)

EPOCH: 1


Loss=0.05030607059597969 Batch_id=234 Accuracy=98.37: 100%|██████████| 235/235 [00:16<00:00, 14.69it/s]



Test set: Average loss: 0.0353, Accuracy: 9883/10000 (98.83%)

EPOCH: 2


...


Test set: Average loss: 0.0231, Accuracy: 9934/10000 (99.34%)

EPOCH: 13


Loss=0.005484098568558693 Batch_id=234 Accuracy=99.29: 100%|██████████| 235/235 [00:16<00:00, 13.96it/s]



Test set: Average loss: 0.0250, Accuracy: 9920/10000 (99.20%)

EPOCH: 14


Loss=0.003590943990275264 Batch_id=234 Accuracy=99.40: 100%|██████████| 235/235 [00:15<00:00, 15.08it/s]



Test set: Average loss: 0.0230, Accuracy: 9930/10000 (99.30%)



In [20]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader_data_aug = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        # transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader_data_aug = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


model =  model_8_maxpool_loc().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader_data_aug, optimizer, epoch)
    test(model, device, test_loader_data_aug)

EPOCH: 0


Loss=0.01958136074244976 Batch_id=234 Accuracy=94.64: 100%|██████████| 235/235 [00:42<00:00,  5.58it/s]



Test set: Average loss: 0.0511, Accuracy: 9838/10000 (98.38%)

EPOCH: 1


Loss=0.0318886898458004 Batch_id=234 Accuracy=98.44: 100%|██████████| 235/235 [00:42<00:00,  5.58it/s]



Test set: Average loss: 0.0329, Accuracy: 9901/10000 (99.01%)

EPOCH: 2


...


Test set: Average loss: 0.0254, Accuracy: 9921/10000 (99.21%)

EPOCH: 13


Loss=0.006261911708861589 Batch_id=234 Accuracy=99.42: 100%|██████████| 235/235 [00:40<00:00,  5.84it/s]



Test set: Average loss: 0.0291, Accuracy: 9909/10000 (99.09%)

EPOCH: 14


Loss=0.004701016005128622 Batch_id=234 Accuracy=99.43: 100%|██████████| 235/235 [00:40<00:00,  5.80it/s]



Test set: Average loss: 0.0244, Accuracy: 9928/10000 (99.28%)



In [23]:
torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader_data_aug = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        # transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                        # transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader_data_aug = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


from torch.optim.lr_scheduler import StepLR

model =  model_8_maxpool_loc().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=0.022303486242890358 Batch_id=234 Accuracy=95.03: 100%|██████████| 235/235 [00:15<00:00, 15.23it/s]



Test set: Average loss: 0.0573, Accuracy: 9815/10000 (98.15%)

EPOCH: 1


Loss=0.028106452897191048 Batch_id=234 Accuracy=98.69: 100%|██████████| 235/235 [00:17<00:00, 13.72it/s]



Test set: Average loss: 0.0331, Accuracy: 9904/10000 (99.04%)

EPOCH: 2


...


Test set: Average loss: 0.0197, Accuracy: 9935/10000 (99.35%)

EPOCH: 13


Loss=0.004785329569131136 Batch_id=234 Accuracy=99.81: 100%|██████████| 235/235 [00:15<00:00, 15.42it/s]



Test set: Average loss: 0.0184, Accuracy: 9936/10000 (99.36%)

EPOCH: 14


Loss=0.008219900541007519 Batch_id=234 Accuracy=99.81: 100%|██████████| 235/235 [00:15<00:00, 15.29it/s]



Test set: Average loss: 0.0194, Accuracy: 9932/10000 (99.32%)



In [27]:
torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader_data_aug = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        # transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                        # transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader_data_aug = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


from torch.optim.lr_scheduler import StepLR

model =  model_8_maxpool_loc().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=0.014161147177219391 Batch_id=234 Accuracy=94.83: 100%|██████████| 235/235 [00:15<00:00, 15.54it/s]



Test set: Average loss: 0.0491, Accuracy: 9848/10000 (98.48%)

EPOCH: 1


Loss=0.0277100820094347 Batch_id=234 Accuracy=98.58: 100%|██████████| 235/235 [00:16<00:00, 14.45it/s]



Test set: Average loss: 0.0302, Accuracy: 9908/10000 (99.08%)

EPOCH: 2


...


Test set: Average loss: 0.0196, Accuracy: 9939/10000 (99.39%)

EPOCH: 13


Loss=0.007659904193133116 Batch_id=234 Accuracy=99.70: 100%|██████████| 235/235 [00:15<00:00, 15.30it/s]



Test set: Average loss: 0.0192, Accuracy: 9940/10000 (99.40%)

EPOCH: 14


Loss=0.0009322818950749934 Batch_id=234 Accuracy=99.72: 100%|██████████| 235/235 [00:15<00:00, 15.25it/s]



Test set: Average loss: 0.0199, Accuracy: 9934/10000 (99.34%)



In [ ]:
+